In [2]:
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import phik
import requests
import streamlit as st
from phik.binning import bin_data
import numpy as np
from sklearn.preprocessing import PowerTransformer, OneHotEncoder, OrdinalEncoder, label_binarize, LabelEncoder 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import (
    mean_absolute_error, 
    r2_score,
    auc,
    mean_absolute_percentage_error,
    make_scorer,
    mean_squared_error,
    accuracy_score,
    balanced_accuracy_score,
    precision_score,
    recall_score,
    roc_curve,
    roc_auc_score,
    f1_score,
    confusion_matrix,
    mean_squared_log_error,
    explained_variance_score,
    median_absolute_error,
    PrecisionRecallDisplay,
    ConfusionMatrixDisplay,
    RocCurveDisplay
)
from catboost import CatBoostClassifier, CatBoostRegressor
import optuna
from xgboost import XGBClassifier, XGBRegressor
import mlflow
import joblib

e:\projects\obesity\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = pd.read_csv(r"E:\projects\obesity\data\obesity.csv")

data.head()

,Gender,Age,Height,Weight,family_history,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,Obesity
0,Female,21.0,162.0,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,152.0,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,180.0,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,180.0,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,178.0,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [4]:
data["Obesity"] = data["Obesity"].map({
    "Insufficient_Weight": 0,
    "Normal_Weight": 1,
    "Overweight_Level_I": 2,
    "Overweight_Level_II": 3,
    "Obesity_Type_I": 4,
    "Obesity_Type_II": 5,
    "Obesity_Type_III": 6
})

In [5]:
y = data["Obesity"]
data.drop(["Obesity"], axis=1, inplace=True)

In [6]:
data.dtypes

Gender             object
Age               float64
Height            float64
Weight            float64
family_history     object
FAVC               object
FCVC              float64
NCP               float64
CAEC               object
SMOKE              object
CH2O              float64
SCC                object
FAF               float64
TUE               float64
CALC               object
MTRANS             object
dtype: object

In [7]:
response = requests.post("http://127.0.0.1:8000/predict_features", json={"data": data.to_dict(orient="records")})

response

<Response [200]>

In [9]:
preds = pd.read_json(response.json()["Answer"], orient="records")

preds

C:\Users\zhora\AppData\Local\Temp\ipykernel_22772\2390793023.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  preds = pd.read_json(response.json()["Answer"], orient="records")


,0
0,1
1,1
2,1
3,2
4,3
...,...
2106,6
2107,6
2108,6
2109,6


In [12]:
preds.rename(columns={0: "Предсказанные значения"})

,Предсказанные значения
0,1
1,1
2,1
3,2
4,3
...,...
2106,6
2107,6
2108,6
2109,6


In [11]:
type(preds)

pandas.core.frame.DataFrame